In [1]:
from datafin.utils import to_ny_time, yesterday, string_formating
from datafin.aws import SecretsClient
from datafin.aws import S3Client

In [10]:
string_formating(to_ny_time(yesterday()).month)

'06'

In [2]:
s = SecretsClient()

In [8]:
pgs3 = S3Client(
    aws_access_key_id=s.get_polygon_aws_key(),
    aws_secret_access_key=s.get_polygon_api_key(),
    is_polygon=True
)

In [11]:


ny_dt = to_ny_time(yesterday())
date_year = string_formating(ny_dt.year)
date_month = string_formating(ny_dt.month)
date_day = string_formating(ny_dt.day)

polygon_minute_aggs_path = f'us_stocks_sip/minute_aggs_v1/{date_year}/{date_month}'
polygon_file_name = f'{date_year}-{date_month}-{date_day}'

df = pgs3.get_csv_compressed(
    path=polygon_minute_aggs_path,
    file_name=polygon_file_name
)